# Assignment 2 — Sudoku CSP Project Code and Documentation

This project was originally implemented as a collection of Python modules available at  
<https://github.com/antonnemch/CP468_A2/>  
It was compiled into this notebook for ease of submission, with one module per code cell.  
An overview of the documentation is found in the table of contents below, and more details are provided in the headers of individual files.

---

## Table of Contents & Documentation

1. **constraints.py** — Binary constraints and geometry relations
2. **sudoku_csp.py** — Sudoku CSP representation  
3. **ac3.py** — AC-3 arc-consistency algorithm  
4. **backtracking.py** — Backtracking search with inference  
5. **heuristics.py** — MRV, degree, and LCV heuristics  
6. **io_utils.py** — Puzzle I/O and console visualization  
7. **run_demo.py** — Automated testing and performance summary  

---

## 1. constraints.py

> **Contributors:** Roop Bassi (code), Anton Nemchinski (documentation)

Defines all **binary constraints** and spatial relations for Sudoku.

### Highlights
- `binary_neq(x1, a, x2, b)` → `a != b` (the sole semantic constraint).  
- `same_row/col/box(x1, x2)` → determine if cells share a unit.  
- Used by `sudoku_csp_from_grid()` to connect neighboring cells.  
- Confirms that **all constraints are binary**, linking pairs of variables only.

---

## 2. sudoku_csp.py

> **Contributors:** Julian Rincon (code), Anton Nemchinski (documentation), Jordan Franschman (neighbor construction fix), Roop Bassi (constraint corrections)

Implements the **Constraint Satisfaction Problem (CSP)** representation for Sudoku.

### Key Design Points
- Each cell is a variable `Var = (row, col)` (0-based).  
- Domains are sets `{1…9}` or singletons `{value}` for given cells.  
- Neighbors are all cells in the same row, column, or 3×3 box.  
- Every constraint is **binary** (`Xi ≠ Xj`), satisfying the assignment requirement.  
- Provides `CSP.all_arcs()` for AC-3 and `CSP.is_solved()` for solution checking.  
- `sudoku_csp_from_grid(grid)` validates input, builds domains and neighbor graph, and returns a `CSP` instance.

---

## 3. ac3.py

> **Contributor:** Jordan Franschman

Implements the **AC-3 (Arc Consistency 3)** algorithm for enforcing local consistency.

### Highlights
- Uses a queue (`deque`) of directed arcs `(Xi, Xj)`.  
- `revise()` removes values from `Xi` with no support in `Xj`.  
- Returns `False` if any domain becomes empty (inconsistency detected).  
- Supports `track_queue=True` to record queue length per iteration for analysis.  
- Used as a pre-processing step and as inference during backtracking.

---

## 4. heuristics.py

> **Contributor:** Jordan Franschman

Implements the **search heuristics** used to improve backtracking efficiency.

### Heuristics
- **MRV (Minimum Remaining Values):** Select variable with fewest possible values.  
- **Degree Tiebreaker:** For ties, choose the variable with most unassigned neighbors.  
- **LCV (Least Constraining Value):** Order values that eliminate the fewest options for neighbors.  

All heuristics are modular and operate directly on the `CSP` structure.

---

## 5. backtracking.py

> **Contributors:** Jordan Franschman (core logic), (Team support)

Provides the **search-based completion** algorithm used when AC-3 alone does not fully solve the puzzle.

### Key Features
- **Trail class** records domain changes for efficient undo (no deep copies).  
- **Forward checking** eliminates inconsistent values from neighbors.  
- Invokes **AC-3 inference** after each assignment → Maintaining Arc Consistency (MAC).  
- Recursively searches until a consistent solution is found or no values remain.  
- Returns boolean status and mutates the `CSP` in place with the solution.

---



## 6. io_utils.py

> **Contributor:** Anton Nemchinski

Handles **file I/O, manual input, and console visualization** for Sudoku puzzles.

### Features
- `read_puzzle(path)` → reads 9×9 text files (`1–9` givens, `0/.'` blanks).  
- `get_valid_puzzles()`, `get_unsolvable_puzzles()`, `get_unofficial_puzzles()` → load datasets by category.  
- `manual_input()` → interactive entry of custom puzzles.  
- `print_grid(grid)` → ASCII board with row/column separators.  
- `print_status(is_consistent, solved)` → standardized result line.  

Serves as the main bridge between the solver and the user interface.

---

## 7. run_demo.py

> **Contributor:** Anton Nemchinski (implementation & integration)

Script for **automated testing and result summaries** across multiple puzzles.

### Capabilities
- **Modes:**  
  - *Short test* → one example per category, prints full AC-3 queue.  
  - *Full test* → runs all puzzles without verbose queue printing.  
  - *Manual mode* → solves user-entered puzzle.  
- Measures runtime and AC-3 iterations (per puzzle and average).  
- Prints explicit “AC-3 → Backtracking” transition messages.  
- Generates final summary including per-file results, counts, and timing statistics.

---

### Summary

Together, these modules form a complete **Sudoku as a Constraint Satisfaction Problem solver**:

1. `sudoku_csp.py` models the problem with binary constraints.  
2. `ac3.py` enforces arc consistency.  
3. `backtracking.py` finishes unsolved cases using MAC search.  
4. `heuristics.py` guides variable/value selection.  
5. `io_utils.py` handles data flow and visualization.  
6. `run_demo.py` benchmarks and reports results.

This implementation satisfies the assignment’s requirements for **binary constraints**, **AC-3 enforcement**, and an **additional complete solving algorithm** while providing clear modular structure and reproducible testing output.



## 1. constraints.py — A* search algorithm


In [25]:

"""
CP468 — constraints.py
Binary constraints and geometric relations for Sudoku.

Contributors:
    - Roop - code
    - Anton - documentation


Functions to Implement:
    - def binary_neq(x1: Var, a: int, x2: Var, b: int) -> bool
        # Returns True iff a != b (Sudoku's pairwise inequality).

    - def same_row(x1: Var, x2: Var) -> bool
        # True if x1 and x2 share the same row.

    - def same_col(x1: Var, x2: Var) -> bool
        # True if x1 and x2 share the same column.

    - def same_box(x1: Var, x2: Var) -> bool
        # True if x1 and x2 share the same 3x3 subgrid (rows//3, cols//3 equal).

Notes:
    - Var is a tuple[int,int] with 0-based indexing.
    - These helpers are used by sudoku_csp.sudoku_csp_from_grid to build neighbors.
"""

def binary_neq(x1, a, x2, b):
    return a != b

def same_row(x1, x2):
    return x1[0] == x2[0]

def same_col(x1, x2):
    return x1[1] == x2[1]

def same_box(x1, x2):
    r1, c1 = x1
    r2, c2 = x2
    return (r1 // 3 == r2 // 3) and (c1 // 3 == c2 // 3)



## 2. sudoku_csp.py — Sudoku as CSP implementation  


In [26]:
"""
CP468 — sudoku_csp.py
CSP definition and Sudoku-specific builder.

Contributors:
    - Julian Rincon - code
    - Anton - documentation
    - Jordan F. - fix neighbor construction
    - Roop - fixed constraints
"""

from __future__ import annotations
from typing import Callable, Dict, Iterable, List, Set, Tuple
#import constraints

Var = Tuple[int, int]  # (row, col), 0-based
Value = int            # 1..9



class CSP:
    """CSP object for Sudoku"""

    def __init__(
        self,
        variables: List[Var],
        domains: Dict[Var, Set[Value]],
        neighbors: Dict[Var, Set[Var]],
        constraint: Callable[[Var, Value, Var, Value], bool],
    ) -> None:
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraint = constraint

    def all_arcs(self) -> Iterable[Tuple[Var, Var]]:
        for xi in self.variables:
            for xj in self.neighbors.get(xi, ()):
                yield (xi, xj)

    def is_solved(self) -> bool:
        for v in self.variables:
            if len(self.domains.get(v, ())) != 1:
                return False
        for xi in self.variables:
            vi = next(iter(self.domains[xi]))
            for xj in self.neighbors.get(xi, ()):
                if xi < xj:
                    vj = next(iter(self.domains[xj]))
                    if not self.constraint(xi, vi, xj, vj):
                        return False
        return True

    def to_grid(self) -> List[List[int]]:
        """Return a 9x9 grid of ints; 0 for unsolved cells"""
        grid: List[List[int]] = [[0 for _ in range(9)] for _ in range(9)]
        for (r, c) in self.variables:
            d = self.domains.get((r, c), set())
            grid[r][c] = next(iter(d)) if len(d) == 1 else 0
        return grid

    def __repr__(self) -> str:
        assigned = sum(1 for v in self.variables if len(self.domains[v]) == 1)
        return (
            f"CSP(vars={len(self.variables)}, "
            f"assigned={assigned}, "
            f"arcs={sum(len(self.neighbors[v]) for v in self.variables)})"
        )


def sudoku_csp_from_grid(grid: List[List[int]]) -> CSP:
    """Construct a Sudoku CSP from a 9x9 integer grid (0=empty)"""
    if len(grid) != 9 or any(len(row) != 9 for row in grid):
        raise ValueError("Grid must be 9x9.")
    for r in range(9):
        for c in range(9):
            v = grid[r][c]
            if not isinstance(v, int) or not (0 <= v <= 9):
                raise ValueError("Grid values must be integers in 0..9.")

    variables: List[Var] = [(r, c) for r in range(9) for c in range(9)]
    full_domain: Set[Value] = set(range(1, 10))

    # Initialize domains
    domains: Dict[Var, Set[Value]] = {}
    for (r, c) in variables:
        val = grid[r][c]
        domains[(r, c)] = {val} if 1 <= val <= 9 else set(full_domain)

    # Build neighbors (row, column, box)
    neighbors: Dict[Var, Set[Var]] = {v: set() for v in variables}
    for (r1, c1) in variables:
        for (r2, c2) in variables:
            if (r1, c1) == (r2, c2):
                continue
            if (same_row((r1, c1), (r2, c2)) or same_col((r1, c1), (r2, c2)) or same_box((r1, c1), (r2, c2))):
                neighbors[(r1, c1)].add((r2, c2))

    return CSP(
        variables=variables,
        domains=domains,
        neighbors=neighbors,
        constraint=binary_neq
    )



## 3. ac3.py — AC-3 algorithm implementation   

In [27]:
"""
CP468 — ac3.py
AC-3 algorithm for enforcing arc consistency on a CSP.

Contributors:
    - Jordan Franschman

Functions:
    - ac3(csp, queue, track_queue) -> (bool, list[int] | None)
        Enforces arc consistency across the CSP using the AC-3 algorithm.
        Iteratively removes unsupported values from variable domains.
    - revise(csp, Xi, Xj) -> bool
        Removes values from Xi’s domain that have no consistent value in Xj.

Description:
    This module ensures that for every variable Xi and each value in its domain,
    there exists at least one value in every neighbor Xj’s domain that satisfies
    the CSP’s binary constraint (in Sudoku, Xi ≠ Xj).

    If any domain becomes empty during propagation, the CSP is inconsistent.
    Otherwise, the process continues until all arcs are consistent.
"""

from collections import deque
from typing import Iterable, Optional

def ac3(csp: CSP, queue: Optional[Iterable[tuple[Var, Var]]] = None, track_queue: bool = False) -> tuple[bool, Optional[list[int]]]:
    """
    Enforce arc consistency using the AC-3 algorithm.
    Args:
        csp: The constraint satisfaction problem instance.
        queue: Optional iterable of arcs (Xi, Xj) to process; if None, use all arcs.
        track_queue: Whether to record the queue length at each step.
    Returns:
        (is_consistent, queue_lengths)
            - is_consistent: True if arc-consistent, False if inconsistency detected.
            - queue_lengths: Optional list of queue sizes per iteration (for visualization).
    """

    # Initialize queue with all arcs if not provided
    arc_queue = deque(queue or csp.all_arcs())
    queue_lengths = [] if track_queue else None

    # Main propagation loop
    while arc_queue:
        # Track queue size evolution if requested
        if track_queue and queue_lengths is not None:
            queue_lengths.append(len(arc_queue))

        # Pop the next arc (Xi, Xj)
        Xi, Xj = arc_queue.popleft()

        # Try to revise Xi’s domain with respect to Xj
        if revise(csp, Xi, Xj):
            # If Xi’s domain is now empty → inconsistency detected
            if len(csp.domains[Xi]) == 0:
                return False, queue_lengths

            # Otherwise, re-enqueue all other neighbors of Xi for further checking
            for Xk in csp.neighbors[Xi]:
                if Xk != Xj:
                    arc_queue.append((Xk, Xi))

    # All arcs processed → CSP is arc-consistent
    return True, queue_lengths


def revise(csp: CSP, Xi: Var, Xj: Var) -> bool:
    """
    Make Xi arc-consistent with respect to Xj.
    Removes any value in Xi’s domain that is unsupported by all values in Xj’s domain.
    Args:
        csp: The constraint satisfaction problem.
        Xi: The source variable.
        Xj: The target variable.
    Returns:
        True if Xi’s domain was revised (values removed), False otherwise.
    """

    revised = False
    domain_Xi = csp.domains[Xi]
    domain_Xj = csp.domains[Xj]
    to_remove = set()

    # For each value in Xi’s domain, check if there exists a supporting value in Xj’s domain
    for x in domain_Xi:
        supported = any(csp.constraint(Xi, x, Xj, y) for y in domain_Xj)
        if not supported:
            to_remove.add(x)
            revised = True

    # Remove unsupported values from Xi’s domain
    domain_Xi -= to_remove

    return revised



## 4. heuristics.py — Heuristic functions (h₁–h₃)


In [28]:
"""
CP468 — heuristics.py
Variable selection and value ordering heuristics for backtracking.

Contributors:
    - Jordan F.

Functions:
    - select_var_mrv(csp) -> Var
    - degree_tiebreak(csp, candidates) -> Var
    - order_values_lcv(csp, var) ->list[int]
"""

from typing import List, Optional
from sudoku_csp import CSP, Var

def select_var_mrv(csp: CSP) -> Optional[Var]:
    """
    Minimum Remaining values heuristic with degree tiebreaker
    Returns unassigned variable with smallest domain and ties are 
     broken by degree heuristic (highest # of constraints on unassigned neighbors).
    """
    
    unassigned = [v for v in csp.variables if len(csp.domains[v]) > 1]
    if not unassigned:
        return None
    
    min_size = min(len(csp.domains[v]) for v in unassigned)
    candidates = [v for v in unassigned if len(csp.domains[v]) == min_size]
    
    if len(candidates) == 1:
        return candidates[0]
    
    return degree_tiebreak(csp, candidates)




def degree_tiebreak(csp: CSP, candidates: List[Var]) -> Var:
    """
    For MRV ties, select var with most unassigned neighbors
    """
    def count_unassigned_neighbors(var: Var) -> int:
        return sum(1 for nb in csp.neighbors[var] if len(csp.domains[nb]) > 1)
    
    return max(candidates,key=count_unassigned_neighbors)


def order_values_lcv(csp: CSP, var: Var) -> List[int]:
    """
    Least Constraining Value heuristic which orders values by how many other domain vlaues they eliminate
    """
    def count_conflicts(value: int) -> int:
        """
        helper to count least number of conflicts
        """
        cons = 0
        for n in csp.neighbors[var]:
            if len(csp.domains[n])> 1 and value in csp.domains[n]:
                cons += 1
        return cons
    return sorted(csp.domains[var], key=count_conflicts)


## 5. backtracking.py — Backtracking algorithm implementation

In [29]:
"""
CP468 — backtracking.py
Additional algorithm when AC-3 does not finish: Backtracking search with optional
forward-checking and/or AC-3 as inference.

Contributors:
    - Jordan F.

Classes:
    - Trail: Efficient undo mechanism for domain changes
    
Functions:
    - solve(csp) -> bool
"""

from typing import Dict, List, Set
from sudoku_csp import CSP, Var
import heuristics
import ac3


class Trail:
    """Efficient undo mechanism for domain changes"""

    def __init__(self):
        self.frames: List[List[tuple[Var, Set[int]]]] = []

    def push_frame(self):
        """Start new backtracking frame"""
        self.frames.append([])

    def record(self, var: Var, removed_values: Set[int]):
        """Record values removed from a variable's domain"""
        if removed_values and self.frames:
            self.frames[-1].append((var, removed_values.copy()))

    def pop_frame_and_undo(self, domains: Dict[Var, Set[int]]):
        """Undo all changes in the current frame"""
        if not self.frames:
            return
        frame = self.frames.pop()
        for var, removed_vals in reversed(frame):
            domains[var] |= removed_vals


def solve(csp: CSP) -> bool:
    """
    Solve CSP using backtracking with AC-3 inference
    """
    trail = Trail()
    return _backtrack(csp, trail)


def _backtrack(csp: CSP, trail: Trail) -> bool:
    if csp.is_solved():
        return True

    var = heuristics.select_var_mrv(csp)
    if var is None:
        return False  # No unassigned variable found but puzzle not solved?

    values = heuristics.order_values_lcv(csp, var)

    for value in values:
        trail.push_frame()
        if _assign_and_infer(csp, var, value, trail):
            if _backtrack(csp, trail):
                return True
        trail.pop_frame_and_undo(csp.domains)

    return False


def _assign_and_infer(csp: CSP, var: Var, value: int, trail: Trail) -> bool:
    """
    Assign a value to a variable and run AC-3 inference.
    Returns False if inconsistency is detected.
    """

    # Record domain changes for var itself
    removed_vals = csp.domains[var] - {value}
    trail.record(var, removed_vals)

    # Assign the value
    csp.domains[var] = {value}

    # Forward checking
    for neighbor in csp.neighbors[var]:
        if value in csp.domains[neighbor]:
            trail.record(neighbor, {value})
            csp.domains[neighbor].remove(value)
            if len(csp.domains[neighbor]) == 0:
                return False
            

    # Run AC-3 on neighbors of var
    arcs = [(neighbor, var) for neighbor in csp.neighbors[var]]
    is_consistent, _ = ac3.ac3(csp, queue=arcs, track_queue=False)

    return is_consistent


## 6. io_utils.py — Functions for ingestion and visualization of puzzles

In [30]:
"""
CP468 — io_utils.py
Reading and writing Sudoku puzzles & console printing helpers for grids and run status.

Contributors:
    - Anton Nemchinski


Functions:


    - def read_puzzle(path: str) -> list[list[int]]
        Format:
            - 9 lines × 9 characters.
            - '1'..'9' for givens.
            - '0' or '.' for blanks.
        Output:
            - 9×9 list of ints (0 for blank).
        Validation:
            - Assert 9 rows; each row has 9 valid characters.
    
    - def get_valid_puzzles() -> list[list[list[int]]]
        Behavior:
            - Returns a list of valid Sudoku puzzles.

    - def get_unsolvable_puzzles() -> list[list[list[int]]]
        Behavior:
            - Returns a list of unsolvable Sudoku puzzles.

    - def get_multiple_solution_puzzles() -> list[list[list[int]]]
        Behavior:
            - Returns a list of puzzles with multiple solutions.

    - def get_all_puzzles() -> list[list[list[int]]]
        Behavior:
            - Returns a combined list of all puzzles from the above three functions.    
    
    - def manual_input() -> list[list[int]]
        Behavior:
            - Prompts user to input a Sudoku puzzle line by line.
            - Validates input format as per read_puzzle.
            - Returns the constructed 9×9 grid.
    
    - def print_grid(grid: list[list[int]]) -> None
        Behavior:
            - Pretty-print the 9x9 grid.
            - Show '.' for 0.
            - Visual separators after cols 3 and 6, and rows 3 and 6.

    - def print_status(*, is_consistent: bool, solved: bool) -> None
        Behavior:
            - Print a concise standardized status line:
              "Arc-consistent: YES/NO | Solved: YES/NO"
"""

from pathlib import Path
from typing import List

def read_puzzle(path: str) -> list[list[int]]:
    grid = []
    with open(path, 'r') as file:
        lines = file.readlines()
        assert len(lines) == 9, "Puzzle must have exactly 9 lines."
        for line in lines:
            line = line.strip()
            assert len(line) == 9, "Each line must have exactly 9 characters."
            row = []
            for char in line:
                if char in '123456789':
                    row.append(int(char))
                elif char in '0.':
                    row.append(0)
                else:
                    raise ValueError(f"Invalid character '{char}' in puzzle.")
            grid.append(row)
    return grid

def _puzzle_dir() -> Path:
    """Return absolute path to the test_puzzles directory. Notebook-safe."""
    here = Path.cwd()
    candidates = [
        here / "test_puzzles",
        here.parent / "test_puzzles",
        here / "CP468_A2" / "test_puzzles",
    ]
    for c in candidates:
        if c.exists():
            return c
    return here / "test_puzzles"

def _load_dir(dir_path: Path, label: str) -> List[Tuple[str, list[list[int]]]]:
    """
    Load every *.txt puzzle in a directory into (label, grid) tuples.
    - label is a POSIX-style relative path from test_puzzles, e.g. "valid/easy01.txt"
    - grid is the 9x9 list[list[int]]
    """
    items: List[Tuple[str, list[list[int]]]] = []
    base = _puzzle_dir()
    if not dir_path.exists():
        print(f"[WARN] Directory not found: {dir_path}")
        return items

    for f in sorted(dir_path.glob("*.txt")):
        if f.is_file():
            rel = f.relative_to(base).as_posix()  # e.g., "valid/foo.txt"
            items.append((rel, read_puzzle(str(f))))

    print(f"[INFO] Loaded {len(items):>2} {label} puzzle(s) from {dir_path.name}/")
    return items

def get_valid_puzzles() -> List[Tuple[str, list[list[int]]]]:
    return _load_dir(_puzzle_dir() / "valid", "valid")

def get_unsolvable_puzzles() -> List[Tuple[str, list[list[int]]]]:
    return _load_dir(_puzzle_dir() / "unsolvable", "unsolvable")

def get_unofficial_puzzles() -> List[Tuple[str, list[list[int]]]]:
    base = _puzzle_dir()
    solved = _load_dir(base / "solved", "solved")
    multi  = _load_dir(base / "multiple_solutions", "multiple-solution")
    return solved + multi

def get_all_puzzles() -> List[Tuple[str, list[list[int]]]]:
    items = get_valid_puzzles() + get_unsolvable_puzzles() + get_unofficial_puzzles()
    print(f"[INFO] Total puzzles loaded: {len(items)}")
    return items

def print_grid(grid: list[list[int]]) -> None:
    """
    Pretty-print the 9x9 grid.
    Show '.' for 0.
    Visual separators after cols 3 and 6, and rows 3 and 6.
    """
    for i, row in enumerate(grid):
        if i > 0 and i % 3 == 0:
            print(" - - - + - - - + - - -")
        row_str = ""
        for j, val in enumerate(row):
            if j > 0 and j % 3 == 0:
                row_str += " |"
            row_str += f" {val if val != 0 else '.'}"
        print(row_str)


def manual_input() -> list[list[int]]:
    """
    Prompt user to input a Sudoku puzzle line by line.
    Validates input format as per read_puzzle.
    Returns the constructed 9×9 grid.
    """
    print("Please enter your Sudoku puzzle line by line.")
    print("Use digits 1-9 for givens and 0 or . for blanks.")
    grid = []
    for i in range(9):
        while True:
            line = input(f"Line {i + 1}: ").strip()
            if len(line) != 9 or any(c not in '1234567890.' for c in line):
                print("Invalid input. Please enter exactly 9 characters using digits 1-9 and 0 or . for blanks.")
                continue
            row = [int(c) if c in '123456789' else 0 for c in line]
            grid.append(row)
            break
    print_grid(grid)
    return grid

def print_status(is_consistent: bool, solved: bool) -> None:
    """
    Print a concise standardized status line:
    "Arc-consistent: YES/NO | Solved: YES/NO"
    """
    consistent_str = "YES" if is_consistent else "NO"
    solved_str = "YES" if solved else "NO"
    print(f"Arc-consistent: {consistent_str} | Solved: {solved_str}")



In [31]:
# --- Notebook-safe patch for io_utils path handling ---

from pathlib import Path

# Optional global override you can set from a notebook cell
_PUZZLE_BASE: Path | None = None

def set_puzzle_base(path: str | Path) -> None:
    """Set the base folder that contains the 'test_puzzles' directory."""
    global _PUZZLE_BASE
    p = Path(path).expanduser().resolve()
    _PUZZLE_BASE = p
    print(f"[io_utils] Puzzle base set to: {p}")

# Monkey-patch _puzzle_dir() to avoid __file__ in notebooks
def _puzzle_dir() -> Path:  # type: ignore[func-override]
    """
    Return absolute path to the test_puzzles directory.
    Notebook-safe: tries (in order)
      1) user-provided _PUZZLE_BASE,
      2) ./test_puzzles next to the notebook,
      3) ../test_puzzles one level up (common when running in a subfolder).
    """
    # 1) Explicit override
    if _PUZZLE_BASE is not None:
        p = (_PUZZLE_BASE / "test_puzzles")
        if p.exists():
            return p

    # 2) ./test_puzzles
    here = Path.cwd()
    p = here / "test_puzzles"
    if p.exists():
        return p

    # 3) ../test_puzzles
    p = here.parent / "test_puzzles"
    if p.exists():
        return p

    raise FileNotFoundError(
        "Could not locate 'test_puzzles' directory. "
        "Use set_puzzle_base(<path-to-folder-containing-test_puzzles>) first."
    )

# Install the monkey patch into io_utils namespace if present
_globs = globals()
if "io_utils" in _globs:
    io_utils._puzzle_dir = _puzzle_dir  # type: ignore[attr-defined]
    io_utils.set_puzzle_base = set_puzzle_base  # type: ignore[attr-defined]
else:
    # If you pasted io_utils directly into the notebook (no module), just ensure
    # these names shadow the originals in the same cell scope.
    pass

print("[io_utils] Notebook-safe _puzzle_dir() installed.")


[io_utils] Notebook-safe _puzzle_dir() installed.


## 7. run_demo.py — Experiment driver & visualization

In [35]:
"""
CP468 — run_demo.py
Demo & benchmarking harness for Sudoku-as-CSP.
Contributors:
    - Anton

Modes:
  - short  : run 1 example from each category; print AC-3 queue contents live
  - full   : run all available puzzles from all categories
  - manual : prompt for a puzzle; print AC-3 queue contents live

Outputs:
  - AC-3 queue trace (in short/manual)
  - Messages when switching from AC-3 to backtracking
  - Runtime per puzzle
  - Summary table with totals + file-by-file results
"""

import argparse
import time
from collections import deque
from typing import Iterable, Optional, List, Tuple

# --------------------------------------------------------------------
# Resolve symbols from prior cells (no imports). Handles both:
#   - bare functions:   ac3(...), solve(...)
#   - module.functions: ac3.ac3(...), backtracking.solve(...)
# --------------------------------------------------------------------
def _resolve_first(candidates: List[Tuple[str, Optional[str]]]):
    g = globals()
    for name, attr in candidates:
        obj = g.get(name)
        if obj is None:
            continue
        if attr:
            cand = getattr(obj, attr, None)
            if callable(cand):
                return cand
        if callable(obj):
            return obj
    raise NameError(f"Could not resolve any callable from {candidates}")

# Expect these to exist from earlier cells:
# - sudoku_csp_from_grid(grid)
# - print_grid(grid), print_status(...)
# - get_valid_puzzles(), get_unsolvable_puzzles(), get_unofficial_puzzles()
# - manual_input()

# Resolve AC-3 (either ac3.ac3 or ac3)
ac3_func = _resolve_first([("ac3", "ac3"), ("ac3", None)])
# Resolve backtracking solve (either backtracking.solve or solve)
solve_func = _resolve_first([("backtracking", "solve"), ("solve", None)])


# ---------- Verbose AC-3 (local) ----------
def ac3_verbose(csp, queue: Optional[Iterable[Tuple[tuple, tuple]]] = None) -> Tuple[bool, int]:
    """AC-3 with live queue CONTENT printing. Returns (is_consistent, pop_count)."""
    arc_queue = deque(queue or csp.all_arcs())
    pops = 0

    def revise(Xi, Xj) -> bool:
        revised = False
        di = csp.domains[Xi]
        dj = csp.domains[Xj]
        to_remove = set()
        for x in di:
            if not any(csp.constraint(Xi, x, Xj, y) for y in dj):
                to_remove.add(x)
        if to_remove:
            di -= to_remove
            revised = True
        return revised

    step = 0
    while arc_queue:
        step += 1
        q_list = list(arc_queue)
        preview = ", ".join([f"{a}->{b}" for (a, b) in q_list[:12]])
        extra = "" if len(q_list) <= 12 else f" ... (+{len(q_list)-12} more)"
        print(f"[AC-3] Step {step:03d} | Queue size={len(q_list)} | {preview}{extra}")

        Xi, Xj = arc_queue.popleft()
        pops += 1

        if revise(Xi, Xj):
            if len(csp.domains[Xi]) == 0:
                print("[AC-3] Domain wipe-out -> inconsistent")
                return False, pops
            for Xk in csp.neighbors[Xi]:
                if Xk != Xj:
                    arc_queue.append((Xk, Xi))
    return True, pops


# ---------- Helper: run one puzzle ----------
def run_one_puzzle(grid: List[List[int]], *, verbose_queue: bool, label: str) -> dict:
    """
    Run AC-3 (verbose or standard), then backtracking if needed.
    Returns a metrics dict.
    """
    metrics = {
        "label": label,
        "ac3_used": True,
        "ac3_pops": 0,
        "ac3_consistent": False,
        "bt_used": False,
        "solved": False,
        "time_sec": 0.0,
        "result_str": "",
    }

    print(f"\n=== Running: {label} ===")
    print_grid(grid)

    t0 = time.perf_counter()
    csp = sudoku_csp_from_grid(grid)

    if verbose_queue:
        print("\n[run] Starting AC-3 (verbose)...")
        consistent, pops = ac3_verbose(csp)
        q_lengths = None
    else:
        print("\n[run] Starting AC-3...")
        # NOTE: The underlying ac3_func may carry strict type hints from another cell.
        #       This call works at runtime; we silence Pylance’s cross-cell type warning:
        consistent, q_lengths = ac3_func(csp, track_queue=True)  # type: ignore[misc,call-arg]
        pops = len(q_lengths) if q_lengths is not None else 0

    metrics["ac3_consistent"] = bool(consistent)
    metrics["ac3_pops"] = int(pops)

    if not consistent:
        metrics["time_sec"] = time.perf_counter() - t0
        metrics["result_str"] = "UNSOLVABLE"
        print_status(is_consistent=False, solved=False)
        print(f"[run] Finished (AC-3 inconsistent). time={metrics['time_sec']:.4f}s")
        return metrics

    if csp.is_solved():
        metrics["solved"] = True
        metrics["time_sec"] = time.perf_counter() - t0
        metrics["result_str"] = "SOLVED BY AC-3"
        print_status(is_consistent=True, solved=True)
        print("\nSolution:")
        print_grid(csp.to_grid())
        print(f"[run] Finished (solved by AC-3). time={metrics['time_sec']:.4f}s")
        return metrics

    print("[run] AC-3 did not finish → switching to Backtracking...")
    metrics["bt_used"] = True

    # Continue from current (partially pruned) CSP
    solved = solve_func(csp)  # type: ignore[misc,call-arg]
    metrics["solved"] = bool(solved)
    metrics["time_sec"] = time.perf_counter() - t0
    metrics["result_str"] = "SOLVED BY BACKTRACKING" if solved else "NO SOLUTION"

    print_status(is_consistent=True, solved=metrics["solved"])
    if solved:
        print("\nSolution:")
        print_grid(csp.to_grid())
    print(f"[run] Finished. time={metrics['time_sec']:.4f}s")

    return metrics


# ---------- Summary printer ----------
def print_summary(results: List[dict]) -> None:
    print("\n==================== SUMMARY ====================")
    total = len(results)
    total_time = sum(r["time_sec"] for r in results)
    avg_time = total_time / total if total else 0.0

    solved = sum(1 for r in results if r["solved"])
    by_ac3 = sum(1 for r in results if "AC-3" in r["result_str"])
    by_bt = sum(1 for r in results if "BACKTRACKING" in r["result_str"])
    unsat = sum(1 for r in results if r["result_str"] == "UNSOLVABLE")

    print(f"Total puzzles run     : {total}")
    print(f"Solved (total)        : {solved}")
    print(f"  - by AC-3 only      : {by_ac3}")
    print(f"  - by Backtracking   : {by_bt}")
    print(f"Unsolvable (AC-3)     : {unsat}")
    print(f"Total runtime (s)     : {total_time:.4f}")
    print(f"Average runtime (s)   : {avg_time:.4f}")
    print("-------------------------------------------------")

    if not results:
        print("(no results)")
        print("=================================================")
        return

    # Dynamically adjust column width for long file paths
    max_label_len = max(len(r["label"]) for r in results)
    print("Per-file results:")
    for r in results:
        label = r["label"]
        status = r["result_str"]
        t = r["time_sec"]
        print(f"  {label:<{max_label_len}} : {status:<22} | {t:.4f} s")

    print("=================================================")


# ---------- Runner modes ----------
def run_short():
    print("\n=== SHORT TEST MODE ===")
    results = []

    # Always one from each core category
    val = get_valid_puzzles()
    uns = get_unsolvable_puzzles()
    unoff = get_unofficial_puzzles()  # list of (label, grid)

    if val:
        label, grid = val[0]
        results.append(run_one_puzzle(grid, verbose_queue=True, label=label))
    if uns:
        label, grid = uns[0]
        results.append(run_one_puzzle(grid, verbose_queue=True, label=label))

    # From unofficial: pick first SOLVED and first MULTIPLE explicitly
    solved_pick = next(((lbl, g) for (lbl, g) in unoff if lbl.startswith("solved/")), None)
    multi_pick  = next(((lbl, g) for (lbl, g) in unoff if lbl.startswith("multiple_solutions/")), None)

    # If you only want one unofficial, prefer multiple_solutions when present
    if multi_pick:
        label, grid = multi_pick
        results.append(run_one_puzzle(grid, verbose_queue=True, label=label))
    elif solved_pick:
        label, grid = solved_pick
        results.append(run_one_puzzle(grid, verbose_queue=True, label=label))

    print_summary(results)



def run_full():
    print("\n=== FULL TEST MODE ===")
    results = []
    all_items = get_valid_puzzles() + get_unsolvable_puzzles() + get_unofficial_puzzles()
    if not all_items:
        print("[info] No puzzles found.")
        print_summary(results)
        return

    for i, (label, grid) in enumerate(all_items, 1):
        print(f"\n=== Puzzle {i}/{len(all_items)} ===")
        results.append(run_one_puzzle(grid, verbose_queue=False, label=label))
    print_summary(results)


def run_manual():
    print("\n=== MANUAL MODE ===")
    grid = manual_input()
    results = [run_one_puzzle(grid, verbose_queue=True, label="manual_input")]
    print_summary(results)


## Experiment Execution and Results

In [36]:
run_short()


=== SHORT TEST MODE ===
[INFO] Loaded 18 valid puzzle(s) from valid/
[INFO] Loaded  7 unsolvable puzzle(s) from unsolvable/
[INFO] Loaded  3 solved puzzle(s) from solved/
[INFO] Loaded  8 multiple-solution puzzle(s) from multiple_solutions/

=== Running: valid/difficult1.txt ===
 . . . | 6 . . | 4 . .
 7 . . | . . 3 | 6 . .
 . . . | . 9 1 | . 8 .
 - - - + - - - + - - -
 . . . | . . . | . . .
 . 5 . | 1 8 . | . . 3
 . . . | 3 . 6 | . 4 5
 - - - + - - - + - - -
 . 4 . | 2 . . | . 6 .
 9 . 3 | . . . | . . .
 . 2 . | . . . | 1 . .

[run] Starting AC-3 (verbose)...
[AC-3] Step 001 | Queue size=1620 | (0, 0)->(4, 0), (0, 0)->(8, 0), (0, 0)->(0, 2), (0, 0)->(0, 5), (0, 0)->(2, 2), (0, 0)->(1, 0), (0, 0)->(0, 8), (0, 0)->(3, 0), (0, 0)->(5, 0), (0, 0)->(0, 1), (0, 0)->(0, 7), (0, 0)->(1, 2) ... (+1608 more)
[AC-3] Step 002 | Queue size=1619 | (0, 0)->(8, 0), (0, 0)->(0, 2), (0, 0)->(0, 5), (0, 0)->(2, 2), (0, 0)->(1, 0), (0, 0)->(0, 8), (0, 0)->(3, 0), (0, 0)->(5, 0), (0, 0)->(0, 1), (0, 0)->

In [ ]:
run_full()


=== FULL TEST MODE ===
[INFO] Loaded 18 valid puzzle(s) from valid/
[INFO] Loaded  7 unsolvable puzzle(s) from unsolvable/
[INFO] Loaded  3 solved puzzle(s) from solved/
[INFO] Loaded  6 multiple-solution puzzle(s) from multiple_solutions/

=== Puzzle 1/34 ===

=== Running: valid/difficult1.txt ===
 . . . | 6 . . | 4 . .
 7 . . | . . 3 | 6 . .
 . . . | . 9 1 | . 8 .
 - - - + - - - + - - -
 . . . | . . . | . . .
 . 5 . | 1 8 . | . . 3
 . . . | 3 . 6 | . 4 5
 - - - + - - - + - - -
 . 4 . | 2 . . | . 6 .
 9 . 3 | . . . | . . .
 . 2 . | . . . | 1 . .

[run] Starting AC-3...
[run] AC-3 did not finish → switching to Backtracking...
Arc-consistent: YES | Solved: YES

Solution:
 5 8 1 | 6 7 2 | 4 3 9
 7 9 2 | 8 4 3 | 6 5 1
 3 6 4 | 5 9 1 | 7 8 2
 - - - + - - - + - - -
 4 3 8 | 9 5 7 | 2 1 6
 2 5 6 | 1 8 4 | 9 7 3
 1 7 9 | 3 2 6 | 8 4 5
 - - - + - - - + - - -
 8 4 5 | 2 1 9 | 3 6 7
 9 1 3 | 7 6 8 | 5 2 4
 6 2 7 | 4 3 5 | 1 9 8
[run] Finished. time=11.3734s

=== Puzzle 2/34 ===

=== Running: va

In [ ]:
run_manual()